# 1. Data Generator
- Raw Data를 읽어옴
- 여기서 만들어진 데이터는 모델의 입력으로 들어감

In [1]:
import os
import numpy as np
import librosa
from tensorflow.keras.utils import Sequence

In [2]:
class RawForVAEGenerator(Sequence):
    def __init__(self, source, wav_dir, files, sourNum='s1', batch_size=10, shuffle=True):
        self.source = source
        self.wav_dir = wav_dir
        self.files = files
        self.sourNum = sourNum
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
        self.sample_rate = 8000
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.source))
        
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __audioread__(self, path, offset=0.0, duration=None, sample_rate=16000):
        signal = librosa.load(path, sr=self.sample_rate, mono=False, offset=offset, duration=duration)

        return signal[0]
    
    def __padding__(self, data):
        n_batch = len(data)
        max_len = max([d.shape[0] for d in data])
        extrapadding = int(np.ceil(max_len / self.sample_rate) * self.sample_rate)
        pad = np.zeros((n_batch, extrapadding))
        
        for i in range(n_batch):
            pad[i, :data[i].shape[0]] = data[i]
        
        return np.expand_dims(pad, -1)
        
    def __data_generation__(self, source_list):
        wav_list = []
        for name in source_list:
            name = name.strip('\n')
            
            s_wav_name = self.wav_dir + self.files + '/' + self.sourNum + '/' + name
            
            # ------- AUDIO READ -------
            s_wav = (self.__audioread__(s_wav_name,  offset=0.0, duration=None, sample_rate=self.sample_rate))
            # --------------------------
            
            # ------- PADDING -------
#             pad_len = max(len(samples1),len(samples2))
#             pad_s1 = np.concatenate([s1_wav, np.zeros([pad_len - len(s1_wav)])])
            
#             extrapadding = ceil(len(pad_s1) / sample_rate) * sample_rate - len(pad_s1)
#             pad_s1 = np.concatenate([pad_s1, np.zeros([extrapadding - len(pad_s1)])])
#             pad_s2 = np.concatenate([s2_wav, np.zeros([extrapadding - len(s2_wav)])])
            # -----------------------
            
            wav_list.append(s_wav)
        
        return wav_list, wav_list, source_list
            
    
    def __len__(self):
        return int(np.floor(len(self.source) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        source_list = [self.source[k] for k in indexes]
        
        if self.files is not 'tt':
            sour, labels, _ = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            label_pad = self.__padding__(labels) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, np.concatenate([label_pad, exp], axis=1)
        else:
            sour, labels, name = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, exp, name

## Data를 어떻게 읽는지에 대한 부분

In [3]:
WAV_DIR = './mycode/wsj0_2mix/use_this/'
LIST_DIR = './mycode/wsj0_2mix/use_this/lists/'

In [4]:
# Directory List file create

wav_dir = WAV_DIR
output_lst = LIST_DIR

for folder in ['tr', 'cv', 'tt']:
    wav_files = os.listdir(wav_dir + folder + '/mix')
    output_lst_files = output_lst + folder + '_wav.lst'
    with open(output_lst_files, 'w') as f:
        for file in wav_files:
            f.write(file + "\n")

print("Generate wav file to .lst done!")

Generate wav file to .lst done!


In [5]:
batch_size = 2

train_dataset = 0
valid_dataset = 0
test_dataset = 0

name_list = []
for files in ['tr', 'cv', 'tt']:
    # --- Lead lst file ---""
    output_lst_files = LIST_DIR + files + '_wav.lst'
    fid = open(output_lst_files, 'r')
    lines = fid.readlines()
    fid.close()
    # ---------------------
    
    if files == 'tr':
        train_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    elif files == 'cv':
        valid_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    else:
        test_batch = 1
        test_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', test_batch)
for a, b in train_dataset:
    print(a.shape)
    print(b.shape)

(2, 64000, 1)
(2, 64001, 1)
(2, 80000, 1)
(2, 80001, 1)


# 2. Building VQ-VAE model with Gumbel Softmax

In [6]:
import threading
from scipy.io.wavfile import write as wav_write
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from tensorflow.keras import backend as Kb
import numpy as np
import pandas as pd
from importlib import reload
import time
from tensorflow.keras.models import Model, Sequential, load_model

In [7]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [8]:
mkdir_p('./CKPT/') # model check point 폴더 만드는 코드
filepath = "./CKPT/CKP_ep_{epoch:d}__loss_{val_loss:.5f}_.h5"

In [9]:
initial_learning_rate = 0.001

# learning rate를 점점 줄이는 부분
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

# validation loss에 대해서 좋은 것만 저장됨
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min'
)

# early stop 하는 부분인데, validation loss에 대해서 제일 좋은 모델이 저장됨
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True
)

In [76]:
class GumbelSoftmax(layers.Layer):
    def __init__(self, temperature=0.5, hard=False, name = 'gumbel_softmax',**kwargs):
        super(GumbelSoftmax, self).__init__(name=name, **kwargs)
        
        self.temperature = temperature
        self.hard = hard
    
    def sample_gumbel(self, shape, eps=1e-20): 
        """Sample from Gumbel(0, 1)"""
        U = tf.random.uniform(shape,minval=0,maxval=1)
        
        return -tf.math.log(-tf.math.log(U + eps) + eps)

    def gumbel_softmax_sample(self, logits, temperature): 
        """ Draw a sample from the Gumbel-Softmax distribution"""
        y = logits + self.sample_gumbel(tf.shape(logits))
        
        return tf.nn.softmax(y / temperature)

    def call(self, inputs):
        y = self.gumbel_softmax_sample(inputs, self.temperature)
        
        if self.hard:
            y_hard = tf.cast(tf.equal(y, tf.math.reduce_max(y, 2, keepdims=True)), y.dtype)
            y = tf.stop_gradient(y_hard - y) + y
        
        return y


class Encoder(layers.Layer):
    def __init__(self, latent_dim, name = 'encoder',**kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        
        self.conv1d_1 = layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='valid', activation='relu')
        self.conv1d_2 = layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='valid', activation='relu')
        self.conv1d_3 = layers.Conv1D(filters=128, kernel_size=3, strides=1, padding='valid', activation='relu')
        self.logit = layers.Conv1D(filters=latent_dim, kernel_size=3, strides=1, activation=None, padding='valid')
    
    def call(self, inputs):
        x = self.conv1d_1(inputs)
        x = self.conv1d_2(x)
        x = self.conv1d_3(x)
        logit = self.logit(x)
        
        return logit


class Decoder(layers.Layer):
    def __init__(self, latent_dim, name = 'decoder',**kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        
        self.trans_conv1d_1 = layers.Conv1DTranspose(filters=128, kernel_size=3, strides=1, activation='relu', padding='valid')
        self.trans_conv1d_2 = layers.Conv1DTranspose(filters=64, kernel_size=3, strides=1, activation='relu', padding='valid')
        self.trans_conv1d_3 = layers.Conv1DTranspose(filters=64, kernel_size=3, strides=1, activation='relu', padding='valid')
        self.logit = layers.Conv1DTranspose(filters=1, kernel_size=3, strides=1, padding='valid', activation=None)
    
    def call(self, inputs):
        x = self.trans_conv1d_1(inputs)
        x = self.trans_conv1d_2(x)
        x = self.trans_conv1d_3(x)
        logit = self.logit(x)
        
        return logit

In [77]:
# Custom Metric Si-sdr

class SiSdr(keras.metrics.Metric):
    def __init__(self, name="Si-sdr", **kwargs):
        super(SiSdr, self).__init__(name=name, **kwargs)
        self.sdr = self.add_weight(name="sdr", initializer="zeros")
        self.count = self.add_weight(name="cnt", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        ori_length = tf.shape(y_true)[1]
        
        # Label & Length divide
        labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
        lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]
        
        # Check sequence length
        batch_size = tf.shape(labels)[0]
        label_size = tf.shape(labels)[1]
        pred_size = tf.shape(y_pred)[1]
        feature_size = tf.shape(labels)[-1]
        
        # Change sequence length
        if label_size < pred_size:
            y_pred = tf.slice(y_pred, [0, 0, 0], [-1, label_size, -1])
        elif label_size > pred_size:
            labels = tf.slice(labels, [0, 0, 0], [-1, pred_size, -1])

        # SI-SDR
        target = tf.linalg.matmul(y_pred, labels, transpose_a=True) * labels / tf.expand_dims(tf.experimental.numpy.square(tf.norm(labels, axis=1)), axis=-1)
        noise = y_pred - target
        values = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
        
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.sdr.assign_add(tf.reduce_sum(values))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.sdr / self.count

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.sdr.assign(0.0)
        self.count.assign(0.0)

In [78]:
# Custom loss

# Custom mse
def custom_mse(y_true, y_pred):
    ori_length = tf.shape(y_true)[1]

    # Label & Length divide
    labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 129]
    lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

    loss = tf.reduce_sum(tf.pow(y_pred - labels, 2), axis=[1, 2])
    loss = tf.reduce_mean(loss)

    return loss


# Custom si-sdr loss
def custom_sisdr_loss(y_true, y_pred):
    ori_length = tf.shape(y_true)[1]

    # Label & Length divide
    labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
    lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

    target = tf.linalg.matmul(y_pred, labels, transpose_a=True) * labels / tf.expand_dims(tf.experimental.numpy.square(tf.norm(labels, axis=1)), axis=-1)
    noise = y_pred - target
    si_sdr = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
    si_sdr = tf.reduce_mean(si_sdr) * -1

    return si_sdr

In [91]:
class Vq_vae(keras.Model):
    def __init__(self, latent_dim, gumbel_hard=False, name='vqvae', **kwargs):
        super(Vq_vae, self).__init__(name=name, **kwargs)
        
        self.latent_dim = latent_dim
        self.softmax = layers.Softmax(-1)
        
        self.encoder = Encoder(latent_dim)
        self.embeddings = layers.Embedding(latent_dim, latent_dim)
        self.decoder = Decoder(latent_dim)
        self.gumbel = GumbelSoftmax(hard=gumbel_hard)
        
    def call(self, inputs, load=False):
        if load:
            inputs = layers.Input(shape=(None, 1))
        
        
        encode = self.encoder(inputs)
        gumbel = self.gumbel(encode)
#         gumbel_argmax = tf.math.argmax(gumbel, axis=-1)
#         embedded = self.embeddings(gumbel_argmax)
        decode = self.decoder(gumbel)
        
        # ------------------ KL loss ------------------
#         qy = self.softmax(encode)
#         log_qy = tf.math.log(qy + 1e-10)
#         log_uniform = qy * (log_qy - tf.math.log(1.0 / self.latent_dim))
#         kl_loss = tf.reduce_sum(log_uniform, axis=[1, 2])
#         kl_loss = tf.reduce_mean(kl_loss) * 0.2
#         # ---------------------------------------------
        
#         self.add_loss(kl_loss)
        
        return encode, decode

# 이렇게 GradientTape 를 사용해서 프로그램 해도 됨

In [92]:
latent_size = 1024
epochs = 300

filePath = "./CKPT/CKP_ep_{0}__loss_{1:.5f}_.h5"
model_path = './CKPT/CKP_ep_145__loss_66.64056_.h5'

loss_fun = custom_mse
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

train_loss = tf.keras.metrics.Mean()
valid_loss = tf.keras.metrics.Mean()
sisdr_Metric = SiSdr()
val_sisdr_Metric = SiSdr()

# Model 불러오는 부분이다
vq_vae = Vq_vae(latent_size, gumbel_hard=False)

In [95]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # Call model
        encode, results = vq_vae(x)
        
        # Calculate losses
        # ------------------ KL loss ------------------
        qy = tf.nn.softmax(encode)
        log_qy = tf.math.log(qy + 1e-10)
        log_uniform = qy * (log_qy - tf.math.log(1.0 / latent_size))
        kl_loss = tf.reduce_sum(log_uniform, axis=[1, 2])
        kl_loss = tf.reduce_mean(kl_loss) * 0.2
        # ---------------------------------------------
        
        loss_value = loss_fun(y, results) + kl_loss
#         loss_value += sum(vq_vae.losses) # Add KL loss
    
    # Update weights
    grads = tape.gradient(loss_value, vq_vae.trainable_weights)
    optimizer.apply_gradients(zip(grads, vq_vae.trainable_weights))
    
    # Update loss and si-sdr
    train_loss.update_state(loss_value)
    sisdr_Metric.update_state(y, results)
    
    return loss_value

@tf.function
def test_step(x, y):
    # Call model
    val_encode, val_results = vq_vae(x)
    
    # Calculate losses
    # ------------------ KL loss ------------------
    qy = tf.nn.softmax(val_encode)
    log_qy = tf.math.log(qy + 1e-10)
    log_uniform = qy * (log_qy - tf.math.log(1.0 / latent_size))
    kl_loss = tf.reduce_sum(log_uniform, axis=[1, 2])
    kl_loss = tf.reduce_mean(kl_loss) * 0.2
    # ---------------------------------------------

    val_loss_value = loss_fun(y, val_results) + kl_loss
    
#     val_loss_value = loss_fun(y, val_results)
#     val_loss_value += sum(vq_vae.losses) # Add KL loss
    
    # Update loss and si-sdr
    valid_loss.update_state(val_loss_value)
    val_sisdr_Metric.update_state(y, val_results)
    
    return val_loss_value

In [96]:
previous_loss = float('inf')

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))
    start_time = time.time()

    # Iterate over the batches of the dataset
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        x_batch_train = tf.cast(x_batch_train, dtype=tf.float32)
        y_batch_train = tf.cast(y_batch_train, dtype=tf.float32)
        
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 1 batches
#         if step % 1 == 0:
#             print("Training loss (for one batch) at step %d: %.4f" % (step, train_loss.result()))
#             print("Training Si-sdr (for one batch) at step %d: %.4f" % (step, sisdr_Metric.result()))
#             print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Run a validation loop at the end of each epoch
    for x_batch_val, y_batch_val in valid_dataset:
        x_batch_val = tf.cast(x_batch_val, dtype=tf.float32)
        y_batch_val = tf.cast(y_batch_val, dtype=tf.float32)
        
        val_loss_value = test_step(x_batch_val, y_batch_val)
    
    print()
    print('----------------------------------------------------------------------------------')
    print("Time taken >>> %.2fs <<<" % (time.time() - start_time))
    print('epoch: {}, Train_loss: {}, Train_Si-sdr: {} \n\
    Valid_loss: {}, Valid_Si-sdr: {}'.format(
        epoch+1,
        train_loss.result(),
        sisdr_Metric.result(),
        valid_loss.result(),
        val_sisdr_Metric.result()))
    print('----------------------------------------------------------------------------------')
    
    # Save Model
    if valid_loss.result() < previous_loss:
        filePath_temp = filePath.format(epoch+1, valid_loss.result())
        
        vq_vae.save_weights(filePath_temp)
        print('Epoch {}: val_loss improved from {} to {}, saving model to {}'.format(
            epoch+1,
            previous_loss,
            valid_loss.result(),
            filePath_temp))
        
        previous_loss = valid_loss.result()
    else:
        print('Epoch {}: val_loss did not improve from {}'.format(
            epoch+1,
            previous_loss))
    print()
    
    # Reset metrics at the end of each epoch
    train_loss.reset_states()
    sisdr_Metric.reset_states()
    valid_loss.reset_states()
    val_sisdr_Metric.reset_states()
    
    # Data shuffle at the end of each epoch
    train_dataset.on_epoch_end()
    valid_dataset.on_epoch_end()


Start of epoch 1

----------------------------------------------------------------------------------
Time taken >>> 13.07s <<<
epoch: 1, Train_loss: 328.7544860839844, Train_Si-sdr: -63.617156982421875 
    Valid_loss: 323.699462890625, Valid_Si-sdr: -50.81320571899414
----------------------------------------------------------------------------------
Epoch 1: val_loss improved from inf to 323.699462890625, saving model to ./CKPT/CKP_ep_1__loss_323.69946_.h5


Start of epoch 2

----------------------------------------------------------------------------------
Time taken >>> 12.98s <<<
epoch: 2, Train_loss: 323.5810546875, Train_Si-sdr: -50.24334716796875 
    Valid_loss: 322.69732666015625, Valid_Si-sdr: -46.6738395690918
----------------------------------------------------------------------------------
Epoch 2: val_loss improved from 323.699462890625 to 322.69732666015625, saving model to ./CKPT/CKP_ep_2__loss_322.69733_.h5


Start of epoch 3

-----------------------------------------


----------------------------------------------------------------------------------
Time taken >>> 11.86s <<<
epoch: 19, Train_loss: 316.38299560546875, Train_Si-sdr: -47.92170715332031 
    Valid_loss: 316.3366394042969, Valid_Si-sdr: -54.21698760986328
----------------------------------------------------------------------------------
Epoch 19: val_loss improved from 316.4400634765625 to 316.3366394042969, saving model to ./CKPT/CKP_ep_19__loss_316.33664_.h5


Start of epoch 20

----------------------------------------------------------------------------------
Time taken >>> 11.74s <<<
epoch: 20, Train_loss: 316.3028564453125, Train_Si-sdr: -52.238914489746094 
    Valid_loss: 316.2586975097656, Valid_Si-sdr: -57.364418029785156
----------------------------------------------------------------------------------
Epoch 20: val_loss improved from 316.3366394042969 to 316.2586975097656, saving model to ./CKPT/CKP_ep_20__loss_316.25870_.h5


Start of epoch 21

------------------------------


----------------------------------------------------------------------------------
Time taken >>> 11.71s <<<
epoch: 37, Train_loss: 315.11590576171875, Train_Si-sdr: -57.09840774536133 
    Valid_loss: 314.96533203125, Valid_Si-sdr: -46.36821746826172
----------------------------------------------------------------------------------
Epoch 37: val_loss improved from 315.049560546875 to 314.96533203125, saving model to ./CKPT/CKP_ep_37__loss_314.96533_.h5


Start of epoch 38

----------------------------------------------------------------------------------
Time taken >>> 11.95s <<<
epoch: 38, Train_loss: 315.0999755859375, Train_Si-sdr: -58.45778274536133 
    Valid_loss: 314.9500732421875, Valid_Si-sdr: -51.80026626586914
----------------------------------------------------------------------------------
Epoch 38: val_loss improved from 314.96533203125 to 314.9500732421875, saving model to ./CKPT/CKP_ep_38__loss_314.95007_.h5


Start of epoch 39

---------------------------------------


----------------------------------------------------------------------------------
Time taken >>> 11.55s <<<
epoch: 56, Train_loss: 314.6303405761719, Train_Si-sdr: -50.452640533447266 
    Valid_loss: 314.5455322265625, Valid_Si-sdr: -47.529422760009766
----------------------------------------------------------------------------------
Epoch 56: val_loss improved from 314.5709228515625 to 314.5455322265625, saving model to ./CKPT/CKP_ep_56__loss_314.54553_.h5


Start of epoch 57

----------------------------------------------------------------------------------
Time taken >>> 11.72s <<<
epoch: 57, Train_loss: 314.6400146484375, Train_Si-sdr: -70.73971557617188 
    Valid_loss: 314.55841064453125, Valid_Si-sdr: -56.617637634277344
----------------------------------------------------------------------------------
Epoch 57: val_loss did not improve from 314.5455322265625


Start of epoch 58

----------------------------------------------------------------------------------
Time taken >>>


----------------------------------------------------------------------------------
Time taken >>> 11.72s <<<
epoch: 76, Train_loss: 314.374755859375, Train_Si-sdr: -54.852989196777344 
    Valid_loss: 314.30224609375, Valid_Si-sdr: -52.30282974243164
----------------------------------------------------------------------------------
Epoch 76: val_loss did not improve from 314.23284912109375


Start of epoch 77

----------------------------------------------------------------------------------
Time taken >>> 11.84s <<<
epoch: 77, Train_loss: 314.32000732421875, Train_Si-sdr: -54.617958068847656 
    Valid_loss: 314.35626220703125, Valid_Si-sdr: -61.63936233520508
----------------------------------------------------------------------------------
Epoch 77: val_loss did not improve from 314.23284912109375


Start of epoch 78

----------------------------------------------------------------------------------
Time taken >>> 11.29s <<<
epoch: 78, Train_loss: 314.2790222167969, Train_Si-sdr: -

...


----------------------------------------------------------------------------------
Time taken >>> 10.72s <<<
epoch: 213, Train_loss: 313.74005126953125, Train_Si-sdr: -51.90026092529297 
    Valid_loss: 313.7108154296875, Valid_Si-sdr: -51.773319244384766
----------------------------------------------------------------------------------
Epoch 213: val_loss did not improve from 313.68927001953125


Start of epoch 214

----------------------------------------------------------------------------------
Time taken >>> 11.19s <<<
epoch: 214, Train_loss: 313.7522888183594, Train_Si-sdr: -57.40985870361328 
    Valid_loss: 313.77117919921875, Valid_Si-sdr: -49.28325653076172
----------------------------------------------------------------------------------
Epoch 214: val_loss did not improve from 313.68927001953125


Start of epoch 215

----------------------------------------------------------------------------------
Time taken >>> 10.91s <<<
epoch: 215, Train_loss: 313.73028564453125, Train


----------------------------------------------------------------------------------
Time taken >>> 11.82s <<<
epoch: 232, Train_loss: 313.692138671875, Train_Si-sdr: -53.41377258300781 
    Valid_loss: 313.7015686035156, Valid_Si-sdr: -53.88176345825195
----------------------------------------------------------------------------------
Epoch 232: val_loss did not improve from 313.6533203125


Start of epoch 233

----------------------------------------------------------------------------------
Time taken >>> 11.93s <<<
epoch: 233, Train_loss: 313.66693115234375, Train_Si-sdr: -56.78913879394531 
    Valid_loss: 313.67364501953125, Valid_Si-sdr: -52.56106948852539
----------------------------------------------------------------------------------
Epoch 233: val_loss did not improve from 313.6533203125


Start of epoch 234

----------------------------------------------------------------------------------
Time taken >>> 11.94s <<<
epoch: 234, Train_loss: 313.71087646484375, Train_Si-sdr: -


----------------------------------------------------------------------------------
Time taken >>> 11.76s <<<
epoch: 252, Train_loss: 313.6776123046875, Train_Si-sdr: -61.405189514160156 
    Valid_loss: 313.6643981933594, Valid_Si-sdr: -55.41044235229492
----------------------------------------------------------------------------------
Epoch 252: val_loss did not improve from 313.64593505859375


Start of epoch 253

----------------------------------------------------------------------------------
Time taken >>> 11.49s <<<
epoch: 253, Train_loss: 313.68341064453125, Train_Si-sdr: -55.11751937866211 
    Valid_loss: 313.6642150878906, Valid_Si-sdr: -54.8149528503418
----------------------------------------------------------------------------------
Epoch 253: val_loss did not improve from 313.64593505859375


Start of epoch 254

----------------------------------------------------------------------------------
Time taken >>> 11.52s <<<
epoch: 254, Train_loss: 313.6312255859375, Train_Si


----------------------------------------------------------------------------------
Time taken >>> 11.40s <<<
epoch: 272, Train_loss: 313.6499328613281, Train_Si-sdr: -67.62318420410156 
    Valid_loss: 313.6755676269531, Valid_Si-sdr: -50.04399490356445
----------------------------------------------------------------------------------
Epoch 272: val_loss did not improve from 313.633544921875


Start of epoch 273

----------------------------------------------------------------------------------
Time taken >>> 11.06s <<<
epoch: 273, Train_loss: 313.6632080078125, Train_Si-sdr: -56.49284362792969 
    Valid_loss: 313.661865234375, Valid_Si-sdr: -53.09614562988281
----------------------------------------------------------------------------------
Epoch 273: val_loss did not improve from 313.633544921875


Start of epoch 274

----------------------------------------------------------------------------------
Time taken >>> 11.20s <<<
epoch: 274, Train_loss: 313.6283874511719, Train_Si-sdr: 


----------------------------------------------------------------------------------
Time taken >>> 10.72s <<<
epoch: 292, Train_loss: 313.64483642578125, Train_Si-sdr: -54.54931640625 
    Valid_loss: 313.6394348144531, Valid_Si-sdr: -63.391204833984375
----------------------------------------------------------------------------------
Epoch 292: val_loss did not improve from 313.62176513671875


Start of epoch 293

----------------------------------------------------------------------------------
Time taken >>> 11.02s <<<
epoch: 293, Train_loss: 313.6361083984375, Train_Si-sdr: -47.4333381652832 
    Valid_loss: 313.6478576660156, Valid_Si-sdr: -54.070648193359375
----------------------------------------------------------------------------------
Epoch 293: val_loss did not improve from 313.62176513671875


Start of epoch 294

----------------------------------------------------------------------------------
Time taken >>> 10.98s <<<
epoch: 294, Train_loss: 313.601806640625, Train_Si-sd

# 여기는 기존의 .fit() 함수를 사용해서 학습하는 부분임

In [90]:
latent_size = 1024
epoch = 100

strategy = tf.distribute.MirroredStrategy(['cpu:0'])
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model_path = './CKPT/CKP_ep_145__loss_66.64056_.h5'
    
    loss_fun = custom_mse
#     loss_fun = custom_sisdr_loss
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=False)

    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    vq_vae.compile(optimizer, loss=loss_fun, metrics=[SiSdr()])
    
    vq_vae(0, True)
    vq_vae.summary()
    
    # 사용 안할 때는 load_model 주석 처리 하자
#     vq_vae.load_weights(model_path)
    # ----------------------------------------
    
    tf.executing_eagerly()

history = vq_vae.fit(
    train_dataset,
    epochs=epoch,
    validation_data=valid_dataset,
    shuffle=True,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_9 (Softmax)          (None, None, 1024)        0         
_________________________________________________________________
encoder (Encoder)            (None, None, 1024)        431552    
_________________________________________________________________
embedding_14 (Embedding)     (None, None, 1024)        1048576   
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           430529    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 1024)        0         
Total params: 1,910,657
Trainable params: 1,910,657
Non-trainable params: 0
_________________________________________________________________
Epoch 

c:\users\moon\anaconda3\envs\study\lib\site-packages\tensorflow\python\keras\metrics.py:257: UserWarning: Metric SiSdr implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


2/2 [==============================] - 15s 9s/step - loss: 325.3329 - Si-sdr: -53.7446 - val_loss: 323.2562 - val_Si-sdr: -57.9471

Epoch 00001: val_loss improved from inf to 323.25623, saving model to ./CKPT\CKP_ep_1__loss_323.25623_.h5
Epoch 2/100
2/2 [==============================] - 11s 8s/step - loss: 323.3943 - Si-sdr: -59.3381 - val_loss: 323.1165 - val_Si-sdr: -45.2328

Epoch 00002: val_loss improved from 323.25623 to 323.11649, saving model to ./CKPT\CKP_ep_2__loss_323.11649_.h5
Epoch 3/100
2/2 [==============================] - 11s 7s/step - loss: 322.7398 - Si-sdr: -49.8750 - val_loss: 320.6661 - val_Si-sdr: -53.1673

Epoch 00003: val_loss improved from 323.11649 to 320.66614, saving model to ./CKPT\CKP_ep_3__loss_320.66614_.h5
Epoch 4/100
2/2 [==============================] - 11s 7s/step - loss: 321.0135 - Si-sdr: -49.8955 - val_loss: 320.2067 - val_Si-sdr: -59.7486

Epoch 00004: val_loss improved from 320.66614 to 320.20673, saving model to ./CKPT\CKP_ep_4__loss_320.2067

2/2 [==============================] - 11s 8s/step - loss: 314.8875 - Si-sdr: -50.8656 - val_loss: 315.0402 - val_Si-sdr: -56.8271

Epoch 00036: val_loss did not improve from 314.92560
Epoch 37/100
2/2 [==============================] - 11s 8s/step - loss: 314.7869 - Si-sdr: -58.4989 - val_loss: 314.7592 - val_Si-sdr: -48.5599

Epoch 00037: val_loss improved from 314.92560 to 314.75919, saving model to ./CKPT\CKP_ep_37__loss_314.75919_.h5
Epoch 38/100
2/2 [==============================] - 11s 8s/step - loss: 314.8262 - Si-sdr: -54.9721 - val_loss: 314.8546 - val_Si-sdr: -52.6129

Epoch 00038: val_loss did not improve from 314.75919
Epoch 39/100
2/2 [==============================] - 12s 7s/step - loss: 314.8690 - Si-sdr: -50.3640 - val_loss: 314.9762 - val_Si-sdr: -61.6639

Epoch 00039: val_loss did not improve from 314.75919
Epoch 40/100
2/2 [==============================] - 11s 8s/step - loss: 314.7933 - Si-sdr: -51.6377 - val_loss: 314.8844 - val_Si-sdr: -48.8283

Epoch 00040: val

2/2 [==============================] - 12s 7s/step - loss: 314.2095 - Si-sdr: -55.3492 - val_loss: 314.1025 - val_Si-sdr: -49.8632

Epoch 00075: val_loss improved from 314.15326 to 314.10248, saving model to ./CKPT\CKP_ep_75__loss_314.10248_.h5
Epoch 76/100
2/2 [==============================] - 12s 8s/step - loss: 314.1237 - Si-sdr: -59.4218 - val_loss: 314.1297 - val_Si-sdr: -57.6949

Epoch 00076: val_loss did not improve from 314.10248
Epoch 77/100
2/2 [==============================] - 11s 7s/step - loss: 314.2234 - Si-sdr: -57.8287 - val_loss: 314.1848 - val_Si-sdr: -56.6849

Epoch 00077: val_loss did not improve from 314.10248
Epoch 78/100
2/2 [==============================] - 12s 7s/step - loss: 314.1364 - Si-sdr: -51.8777 - val_loss: 314.0632 - val_Si-sdr: -51.9020

Epoch 00078: val_loss improved from 314.10248 to 314.06317, saving model to ./CKPT\CKP_ep_78__loss_314.06317_.h5
Epoch 79/100
2/2 [==============================] - 12s 8s/step - loss: 314.0662 - Si-sdr: -55.4729 -

## 2.2. Encoder 부르는 방법, Decoder에 값 넣는 방법

In [50]:
latent_size = 512
epoch = 200
BATCH_SIZE = 2

strategy = tf.distribute.MirroredStrategy(['cpu:0'])
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model_path = './CKPT/CKP_ep_283__loss_141.77045_.h5'
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=False)
    vq_vae(0, True)
    vq_vae.summary()
    
    vq_vae.load_weights(model_path)
    
    # 이렇게 하면, transforer의 input으로 들어가는 one-hot 형식의 값을 얻을 수 있음
    for inputs, label in train_dataset:
        encode = vq_vae.encoder(inputs).numpy()
        encode_onehot = tf.cast(tf.equal(encode, tf.math.reduce_max(encode, 2, keepdims=True)), encode.dtype)
    
    # 이렇게 하면, transformer의 output을 vq-vae의 decoder 입력으로 넣을 수 있음
    for inputs, label in train_dataset:
        encode = vq_vae.encoder(inputs).numpy()
        encode_onehot = tf.cast(tf.equal(encode, tf.math.reduce_max(encode, 2, keepdims=True)), encode.dtype)
        
        # 이렇게 이전 layer의 출렫을 넣으면 됨
        decode = vq_vae.decoder(encode_onehot).numpy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_25 (Softmax)         (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________


# 3. Test Model

In [51]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [52]:
mkdir_p('./test_wav/') # Result wav 폴더 만드는 코드

In [53]:
def audiowrite(data, path, samplerate=16000, normalize=False, threaded=True):
    """ Write the audio data ``data`` to the wav file ``path``
    The file can be written in a threaded mode. In this case, the writing
    process will be started at a separate thread. Consequently, the file will
    not be written when this function exits.
    :param data: A numpy array with the audio data
    :param path: The wav file the data should be written to
    :param samplerate: Samplerate of the audio data
    :param normalize: Normalize the audio first so that the values are within
        the range of [INTMIN, INTMAX]. E.g. no clipping occurs
    :param threaded: If true, the write process will be started as a separate
        thread
    :return: The number of clipped samples
    """
    data = data.copy()
    int16_max = np.iinfo(np.int16).max
    int16_min = np.iinfo(np.int16).min

    if normalize:
        if not data.dtype.kind == 'f':
            data = data.astype(np.float)
        data /= np.max(np.abs(data))

    if data.dtype.kind == 'f':
        data *= int16_max

    sample_to_clip = np.sum(data > int16_max)
    if sample_to_clip > 0:
        print('Warning, clipping {} samples'.format(sample_to_clip))
    data = np.clip(data, int16_min, int16_max)
    data = data.astype(np.int16)

    if threaded:
        threading.Thread(target=wav_write, args=(path, samplerate, data)).start()
    else:
        wav_write(path, samplerate, data)

    return sample_to_clip

In [57]:
with tf.device('/cpu:0'):
    latent_size = 512
    sample_rate = 8000
    model_path = './CKPT/CKP_ep_283__loss_141.77045_.h5'
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=True)
    vq_vae(0, True)
    vq_vae.summary()
    vq_vae.load_weights(model_path)

    for batch in test_dataset:
        input_batch, length_batch, name = batch

        result = vq_vae.predict(input_batch)
        
        wav_name = './test_wav/' + name[0][:-5] + '_s1.wav'
        audiowrite(result[0], wav_name, sample_rate, True, True)

Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_29 (Softmax)         (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________


# 여기 밑에는 연습장임

In [63]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding

model = Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(None, 1)))
model.add(layers.Conv1D(filters=512, kernel_size=3, padding='same'))

input_array = np.random.randn(2, 10, 1)
input_array2 = np.random.randn(2, 9, 1)
with tf.device('/cpu:0'):
    model.compile('rmsprop', 'mse')

    output_array = model.predict(input_array)
    output_array2 = model.predict(input_array2)

In [64]:
tf.reduce_sum(output_array, axis=[1, 2])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-3.2364433, -5.300486 ], dtype=float32)>

In [65]:
print(output_array)
print(output_array.shape)
print(output_array2.shape)

[[[ 0.00213541  0.01945671 -0.01550578 ... -0.00601448  0.01089236
    0.03509172]
  [-0.00979825 -0.01161936 -0.03782888 ... -0.01964907  0.02604836
   -0.01746842]
  [-0.01680236 -0.04395067  0.00300831 ...  0.02191263  0.02651824
   -0.0194194 ]
  ...
  [ 0.03252641 -0.03450746 -0.01116706 ...  0.04157236  0.00767434
    0.00222839]
  [ 0.03689077 -0.00653183  0.0138604  ...  0.03399033  0.01157293
   -0.01727439]
  [ 0.00956812  0.00808328 -0.00985544 ...  0.02565406  0.01499317
    0.0055492 ]]

 [[ 0.00062903 -0.02250458 -0.01209254 ... -0.00480657  0.02518445
    0.00395943]
  [ 0.00067916 -0.03764667 -0.00435403 ... -0.00190964  0.0198026
   -0.01638822]
  [ 0.03713017 -0.02867446 -0.01009638 ...  0.03275302  0.02717083
    0.00666822]
  ...
  [ 0.01911856  0.02970011 -0.00635595 ... -0.01212452  0.03101327
   -0.01940764]
  [-0.04032427 -0.04538781 -0.00670558 ...  0.00985033  0.0156134
   -0.02115321]
  [ 0.0188317  -0.04442241  0.00422117 ...  0.03525497 -0.0081987
   -0.036

In [69]:
one_hot = tf.math.argmax(tf.cast(tf.equal(output_array, tf.math.reduce_max(output_array, 2, keepdims=True)), output_array.dtype), axis=-1)
print(one_hot.shape)
print(one_hot)
# layers.Embedding(512, 512)(one_hot)

(2, 10)
tf.Tensor(
[[228 249 283 206  20 206 435  32 270  30]
 [428 206  20 244 357 289 324 249 498 134]], shape=(2, 10), dtype=int64)


In [70]:
one_hot = tf.math.argmax(output_array, axis=-1)
print(one_hot.shape)
print(one_hot)

(2, 10)
tf.Tensor(
[[228 249 283 206  20 206 435  32 270  30]
 [428 206  20 244 357 289 324 249 498 134]], shape=(2, 10), dtype=int64)


In [50]:
qy = tf.nn.softmax(output_array)
log_qy = tf.math.log(qy + 1e-10)
log_uniform = qy * (log_qy - tf.math.log(1.0 / 512))
kl_loss = tf.reduce_sum(log_uniform, axis=[1, 2])
kl_loss = tf.reduce_mean(kl_loss)

print(kl_loss)

tf.Tensor(0.012074914, shape=(), dtype=float32)


In [30]:
batch_size = tf.shape(output_array)[0]
array1_size = tf.shape(output_array)[1]
array2_size = tf.shape(output_array2)[1]
feature_size = tf.shape(output_array)[-1]

if array1_size < array2_size:
#     append_size = array1_size - array2_size
#     append_zeros = tf.zeros([batch_size, append_size, feature_size])
#     append_zeros = tf.Variable(initial_value=tf.zeros((batch_size, append_size, feature_size)))
#     output_array2 = tf.concat([output_array2, append_zeros], axis=1)
    output_array2 = tf.slice(output_array2, [0, 0, 0], [-1, array1_size, -1])
elif array1_size > array2_size:
#     append_size = array2_size - array1_size
#     append_zeros = tf.zeros([batch_size, append_size, feature_size])
#     append_zeros = tf.Variable(initial_value=tf.zeros((batch_size, append_size, feature_size)))
#     output_array = tf.concat([output_array, append_zeros], axis=1)
    output_array = tf.slice(output_array, [0, 0, 0], [-1, array2_size, -1])

print(output_array.shape)
print(output_array2.shape)
# output_array0 = output_array[1]
# output_array20 = output_array2[1]
# target = np.sum(output_array20 * output_array0) * output_array0 / np.square(np.linalg.norm(output_array0, ord=2))
# noise = output_array20 - target
# npnp = 10 * np.log10(np.square(np.linalg.norm(target, ord=2)) / np.square(np.linalg.norm(noise, ord=2)))
# print(npnp)

target = tf.linalg.matmul(output_array2, output_array, transpose_a=True) * output_array / tf.expand_dims(tf.experimental.numpy.square(tf.norm(output_array, axis=1)), axis=-1)
noise = output_array2 - target
si_sdr = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
si_sdr = tf.reduce_mean(si_sdr)
print(si_sdr)

(2, 9, 1)
(2, 9, 1)
tf.Tensor(2.8309882, shape=(), dtype=float32)


In [20]:
tf.cast(tf.equal(output_array, tf.math.reduce_max(output_array, 2, keepdims=True)), output_array.dtype)

<tf.Tensor: shape=(2, 9, 4), dtype=float32, numpy=
array([[[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)>

In [21]:
output_array

<tf.Tensor: shape=(2, 9, 4), dtype=float32, numpy=
array([[[-0.03009652, -0.03612775, -0.06680483, -0.03670201],
        [-0.04768711, -0.12344762, -0.03924457, -0.11762322],
        [ 0.01808495, -0.16106637, -0.19467078, -0.15282159],
        [-0.0986427 , -0.08625205, -0.12661007, -0.16366175],
        [-0.09758376, -0.08886974, -0.0433558 , -0.19985165],
        [-0.06933096, -0.03154394, -0.13725929, -0.20143284],
        [ 0.03375649,  0.00182091, -0.01022564, -0.35924646],
        [-0.01645333, -0.10466891, -0.13975918, -0.12066491],
        [-0.13588801, -0.08173112, -0.00253745, -0.28615874]],

       [[ 0.04865369, -0.02880372, -0.06414615, -0.07730438],
        [-0.08225074, -0.03192509, -0.06216412, -0.08035193],
        [-0.09515338,  0.04221668,  0.14230826, -0.23082384],
        [-0.00094383,  0.05597762, -0.09290768, -0.08630683],
        [-0.09894791, -0.04727853, -0.01004983, -0.30325216],
        [ 0.01705559, -0.16948727, -0.08829505, -0.16453639],
        [-0.07230

In [67]:
output_softmax = tf.nn.softmax(output_array)
output_softmax

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0.24628553, 0.282701  , 0.2385324 , 0.23248109],
        [0.2298986 , 0.23856457, 0.25392184, 0.27761498],
        [0.2101039 , 0.2444843 , 0.26824066, 0.27717113],
        [0.22202027, 0.29728216, 0.2501223 , 0.23057525],
        [0.24084595, 0.2724257 , 0.25000373, 0.23672463]],

       [[0.23988546, 0.27957046, 0.23915865, 0.24138539],
        [0.24975686, 0.27135593, 0.24479878, 0.23408844],
        [0.24278015, 0.26340333, 0.24202275, 0.25179377],
        [0.23139507, 0.262904  , 0.25970972, 0.2459912 ],
        [0.2601803 , 0.25627998, 0.24524413, 0.23829558]]], dtype=float32)>

In [73]:
output_reshape = tf.reshape(output_softmax, [-1, 4])
output_reshape.shape

TensorShape([10, 4])

In [83]:
tf.reshape(tf.nn.softmax(output_array), [-1, 5, 4])

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0.24628553, 0.282701  , 0.2385324 , 0.23248109],
        [0.2298986 , 0.23856457, 0.25392184, 0.27761498],
        [0.2101039 , 0.2444843 , 0.26824066, 0.27717113],
        [0.22202027, 0.29728216, 0.2501223 , 0.23057525],
        [0.24084595, 0.2724257 , 0.25000373, 0.23672463]],

       [[0.23988546, 0.27957046, 0.23915865, 0.24138539],
        [0.24975686, 0.27135593, 0.24479878, 0.23408844],
        [0.24278015, 0.26340333, 0.24202275, 0.25179377],
        [0.23139507, 0.262904  , 0.25970972, 0.2459912 ],
        [0.2601803 , 0.25627998, 0.24524413, 0.23829558]]], dtype=float32)>

In [76]:
# tf.cast(tf.equal(y, tf.reduce_max(y,1,keep_dims=True)), y.dtype)
output_hard = tf.cast(tf.equal(output_reshape, tf.math.reduce_max(output_reshape, 1, keepdims=True)), output_softmax.dtype)
output_hard

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)>

In [81]:
tf.reshape(output_hard, [-1, 5, 4])

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]], dtype=float32)>

In [322]:
tf.transpose(output_array, perm=[0, 2, 1]).shape

TensorShape([2, 4, 5])

In [316]:
layers.Softmax(output_array)

In [317]:
np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]).shape

(3, 3)

In [65]:
np.log(10)

2.302585092994046